# RipeSens: Deteksi Kematangan Buah Berbasis Sensor Gas dan Machine Learning

Sistem ini dikembangkan untuk mendeteksi tingkat kematangan buah secara otomatis dengan memanfaatkan sensor gas dan mikrokontroler ESP32. Sensor mendeteksi emisi gas ethylene dari buah dalam wadah tertutup, lalu mengirim data ke platform digital untuk dianalisis. Tujuannya adalah memberikan solusi pemantauan kematangan buah yang lebih akurat, efisien, dan dapat dilakukan secara jarak jauh dibanding metode manual. Sistem ini diintegrasikan dengan model machine learning untuk memprediksi kematangan buah berdasarkan pola data sensor.

## - Import Library

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
import tensorflow as tf
from tensorflow.keras.models import load_model

## - Import Dataset

Dataset diambil dari [Archive ICS](https://archive.ics.uci.edu/dataset/1081/gas%2Bsensor%2Barray%2Blow-concentration?utm_source=chatgpt.com). Isi dataset mengandung beberapa gabungan dari sensor, yaitu TGS2603, TGS2630, TGS813, TGS822, MQ-135, MQ-137, MQ-138, 2M012, VOCS-P, dan 2SH12. Hasil bacaan tiap sensor sebanyak 900 kolom, contohnya di kolom sampel 900 pertama adalah hasil bacaan sensor TGS2603. Dataset ini digunakan karena memiliki kemiripan dalam pengambilan data sensor gas meskipun bukan mengambil dari buah langsung.

In [31]:
url = "https://drive.google.com/uc?id=1bCaWXaiTrf_Z0_6TrCzIxtYco8S_0x1b"
df = pd.read_csv(url)

In [32]:
df.head(10)

,ethanol,100ppb,0.3565,0.3345,0.3575,0.333,0.3565.1,0.3345.1,0.3565.2,0.3355,...,3.93.74,3.929.105,3.929.106,3.929.107,3.9315.53,3.935.12,3.9275.108,3.93.75,3.9325.24,3.9315.54
0,ethanol,100ppb,0.3525,0.3305,0.3525,0.3320,0.3550,0.3305,0.3525,0.3280,...,3.9190,3.9190,3.9180,3.9180,3.9180,3.9180,3.9190,3.9180,3.9180,3.9205
1,ethanol,100ppb,0.3550,0.3345,0.3540,0.3320,0.3550,0.3330,0.3550,0.3320,...,3.8815,3.8840,3.8800,3.8800,3.8840,3.8840,3.8815,3.8840,3.8825,3.8815
2,ethanol,100ppb,0.3415,0.3220,0.3430,0.3210,0.3430,0.3235,0.3415,0.3210,...,3.9460,3.9485,3.9410,3.9470,3.9460,3.9425,3.9460,3.9460,3.9460,3.9460
3,ethanol,100ppb,0.3500,0.3245,0.3465,0.3320,0.3455,0.3245,0.3490,0.3235,...,3.9375,3.9400,3.9385,3.9375,3.9375,3.9365,3.9400,3.9400,3.9365,3.9365
4,ethanol,200ppb,0.3380,0.3220,0.3365,0.3345,0.3380,0.3365,0.3380,0.3210,...,3.8730,3.8705,3.8715,3.8715,3.8730,3.8705,3.8715,3.8740,3.8680,3.8730
5,ethanol,200ppb,0.3405,0.3405,0.3415,0.3260,0.3405,0.3210,0.3380,0.3235,...,3.8730,3.8730,3.8730,3.8740,3.8715,3.8730,3.8730,3.8750,3.8750,3.8715
6,ethanol,200ppb,0.3455,0.3440,0.3455,0.3430,0.3475,0.3455,0.3465,0.3280,...,3.8705,3.8690,3.8690,3.8715,3.8690,3.8740,3.8715,3.8705,3.8715,3.8690
7,ethanol,200ppb,0.3455,0.3440,0.3455,0.3305,0.3465,0.3345,0.3465,0.3295,...,3.8775,3.8740,3.8765,3.8775,3.8775,3.8765,3.8765,3.8765,3.8790,3.8775
8,ethanol,200ppb,0.3490,0.3330,0.3500,0.3465,0.3475,0.3295,0.3490,0.3475,...,3.8680,3.8640,3.8690,3.8680,3.8640,3.8680,3.8640,3.8680,3.8665,3.8665
9,ethanol,50ppb,0.3550,0.3345,0.3550,0.3355,0.3540,0.3345,0.3540,0.3355,...,3.9275,3.9290,3.9300,3.9290,3.9275,3.9315,3.9255,3.9275,3.9315,3.9315


In [33]:
print(df.iloc[:, 0].unique())
print(df.iloc[:, 1].unique())

['ethanol' 'acetone' 'toluene' 'ethyl acetate' 'isopropanol' 'hexane']
['100ppb' '200ppb' '50ppb']


Gas yang akan diambil adalah hexane, karena memiliki struktur kimia yang mirip dengan etilen.

In [34]:
df_hexane = df[df.iloc[:, 0] == 'hexane'].copy()
df_hexane.head(15)

,ethanol,100ppb,0.3565,0.3345,0.3575,0.333,0.3565.1,0.3345.1,0.3565.2,0.3355,...,3.93.74,3.929.105,3.929.106,3.929.107,3.9315.53,3.935.12,3.9275.108,3.93.75,3.9325.24,3.9315.54
74,hexane,100ppb,0.3855,0.3660,0.3845,0.3855,0.3830,0.3685,0.3855,0.3660,...,3.7800,3.7750,3.7750,3.7750,3.7765,3.7750,3.7765,3.7765,3.7765,3.7740
75,hexane,100ppb,0.3855,0.3695,0.3855,0.3660,0.3855,0.3735,0.3845,0.3720,...,3.8190,3.8190,3.8190,3.8190,3.8165,3.8165,3.8215,3.8215,3.8205,3.8155
76,hexane,100ppb,0.3845,0.3675,0.3845,0.3720,0.3820,0.3675,0.3845,0.3695,...,3.8360,3.8350,3.8335,3.8360,3.8335,3.8335,3.8360,3.8360,3.8350,3.8360
77,hexane,100ppb,0.3170,0.3060,0.3195,0.3050,0.3185,0.3195,0.3185,0.3015,...,3.8945,3.8900,3.8900,3.8945,3.8945,3.8910,3.8935,3.8935,3.8900,3.8925
78,hexane,100ppb,0.3160,0.3150,0.3135,0.2965,0.3160,0.2965,0.3160,0.2990,...,3.8885,3.8885,3.8860,3.8900,3.8875,3.8885,3.8900,3.8885,3.8860,3.8875
79,hexane,200ppb,0.3260,0.3060,0.3235,0.3050,0.3245,0.3050,0.3235,0.3050,...,3.8740,3.8740,3.8740,3.8730,3.8740,3.8740,3.8775,3.8730,3.8750,3.8750
80,hexane,200ppb,0.3405,0.3220,0.3390,0.3260,0.3390,0.3195,0.3405,0.3195,...,3.8435,3.8460,3.8435,3.8420,3.8460,3.8445,3.8410,3.8470,3.8485,3.8420
81,hexane,200ppb,0.3525,0.3380,0.3540,0.3330,0.3525,0.3355,0.3525,0.3430,...,3.8140,3.8140,3.8180,3.8190,3.8140,3.8180,3.8155,3.8140,3.8155,3.8155
82,hexane,200ppb,0.3575,0.3390,0.3565,0.3380,0.3575,0.3365,0.3565,0.3390,...,3.8105,3.8115,3.8095,3.8095,3.8080,3.8115,3.8115,3.8070,3.8055,3.8130
83,hexane,200ppb,0.3760,0.3625,0.3760,0.3610,0.3760,0.3610,0.3785,0.3660,...,3.7960,3.7960,3.7995,3.7995,3.7985,3.7995,3.7995,3.7970,3.7970,3.7970


Dari sensor yang ada, kita hanya akan mengambil data sensor TGS813 karena kemiripannya dengan sensor MQ-4, yaitu sensor yang dipakai pada alat yang sudah dibuat.

## - Training Model

In [35]:
map_label = {'50ppb': 'unripe', '100ppb': 'midripe', '200ppb': 'ripe'}
ripeness_labels = df_hexane.iloc[:, 1].map(map_label).to_numpy() #

le = LabelEncoder()
X = df_hexane.iloc[:, 1802:2702] #TGS813 ada diurutan ketiga, artinya di sampel 900 yang ketiga (sampel 900 pertama ada di kolom 2 hinnga kolom 901)
y = le.fit_transform(ripeness_labels)

In [36]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [38]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')  # 3 kelas: unripe, midripe, ripe
])

model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [39]:
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test))

Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.4167 - loss: 1.5364 - val_accuracy: 0.3333 - val_loss: 1.3012
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 111ms/step - accuracy: 0.3333 - loss: 0.9575 - val_accuracy: 0.6667 - val_loss: 1.3611
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 140ms/step - accuracy: 0.3333 - loss: 0.8564 - val_accuracy: 0.6667 - val_loss: 1.3013
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 98ms/step - accuracy: 0.5833 - loss: 0.8152 - val_accuracy: 0.6667 - val_loss: 1.2177
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.5833 - loss: 0.7830 - val_accuracy: 0.6667 - val_loss: 1.1632
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 102ms/step - accuracy: 0.5000 - loss: 0.7238 - val_accuracy: 0.6667 - val_loss: 1.1011
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 137ms/step - accuracy: 0.7500 - loss: 0.6880 - val_accuracy: 0.6667 - val_loss: 1.0179
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 104ms/step - accuracy: 0.6667 - loss: 0.6367 - val_accuracy: 0.6667 - val_loss: 0.9

In [40]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Akurasi model:", accuracy)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.3333 - loss: 6.3744
Akurasi model: 0.3333333432674408


In [41]:
model.save("ripeness_model_tgs813.h5")

## - Inferensi

In [42]:
model = load_model("ripeness_model_tgs813.h5")

# Ambil 1 sampel dari data uji
sample_input = X_test[0].reshape(1, -1)

# Prediksi menggunakan model yang sudah dilatih
prediction = model.predict(sample_input)
predicted_class = np.argmax(prediction)

# Konversi angka ke label nama
predicted_label = le.inverse_transform([predicted_class])[0]

print("Prediksi tingkat kematangan buah:", predicted_label)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 123ms/step
Prediksi tingkat kematangan buah: midripe
